# LSTM Model


## Import Data
### Players from RPS_game.py

In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPS_encode, RPS_decode, RPS_encode_dataframe
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join

## Read and Concatenate Games
Take all games and load into dataframe

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

50000


,Player,Opponent,Result,Winning_Play
0,S,P,1,S
1,S,P,1,S
2,P,R,1,P
3,P,P,0,S
4,R,S,1,R


# LSTM Training
### Parameters

In [3]:
seq_length = 100 #how far back the ltsm looks when training
#Model parameters
BATCH_SIZE = 64
VOCAB_SIZE = 3 # RPS
EMBEDDING_DIM = 8
RNN_UNITS = 512

TRAINING_EPOCHS = 200

#Kris_Mrugesh settings
#seq_length = 100 
#BATCH_SIZE = 64
#VOCAB_SIZE = 3 
#EMBEDDING_DIM = 128
#RNN_UNITS = 512
#TRAINING_EPOCHS = 300


### Format data for LTSM training

In [4]:
temp = games_concat.drop("Result", axis = 1)
temp = RPS_encode_dataframe(temp)
input_data = tf.data.Dataset.from_tensor_slices(temp["Player"])
target_data = tf.data.Dataset.from_tensor_slices(temp["Winning_Play"])
print(temp)
print(input_data)

#split into sequences
input_sequences = input_data.batch(seq_length+1, drop_remainder = True)
target_sequences = target_data.batch(seq_length+1, drop_remainder = True)
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)

dataset = tf.data.Dataset.zip((inputs, targets))
for element in dataset:
    print(element)
    break


     Player  Opponent  Winning_Play
0         2         1             2
1         2         1             2
2         1         0             1
3         1         1             2
4         0         2             0
..      ...       ...           ...
995       2         1             2
996       2         1             2
997       1         0             1
998       2         1             2
999       0         2             0

[50000 rows x 3 columns]
<TensorSliceDataset shapes: (), types: tf.int64>
<BatchDataset shapes: (101,), types: tf.int64>
(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([2, 2, 1, 1, 0, 0, 2, 2, 1, 1, 1, 0, 2, 1, 0, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 0, 1, 2, 2, 0, 0, 0, 2, 0, 2, 0, 1, 0, 2, 0, 0, 1, 1, 2, 2,
       0, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 1, 0,
       1, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 0, 2, 2, 1, 0, 1, 1,
       2, 0, 1, 1, 0, 2, 2, 0, 2, 0, 0, 2], dtype=int64)>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
ar

### Build the model

In [5]:
#Shuffle sequences before batching
data = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder = True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]))
    model.add(LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size+1))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 8)             24        
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4231168   
_________________________________________________________________
dense (Dense)                (64, None, 4)             4100      
Total params: 4,235,292
Trainable params: 4,235,292
Non-trainable params: 0
_________________________________________________________________


### Create loss function

In [6]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

### Compile the model

In [7]:
model.compile(optimizer='adam', loss=loss)

### Create Checkpoints
configure the model to create checkpoints as it trains so the model can be reloaded and continue training

In [8]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True)

print(tf.__version__)

2.3.1


In [10]:
#CALL TO LOAD LATEST CHECKPOINT FOR TRAINING
checkpoint_dir = './training_checkpoints'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

### Training the model

In [10]:
history = model.fit(data, epochs = TRAINING_EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/200
7/7 [==============================] - 0s 42ms/step - loss: 0.5328
Epoch 2/200
7/7 [==============================] - 0s 40ms/step - loss: 0.5331
Epoch 3/200
7/7 [==============================] - 0s 39ms/step - loss: 0.5231
Epoch 4/200
7/7 [==============================] - 0s 40ms/step - loss: 0.5183
Epoch 5/200
7/7 [==============================] - 0s 41ms/step - loss: 0.4996
Epoch 6/200
7/7 [==============================] - 0s 38ms/step - loss: 0.5101
Epoch 7/200
7/7 [==============================] - 0s 38ms/step - loss: 0.5091
Epoch 8/200
7/7 [==============================] - 0s 39ms/step - loss: 0.5140
Epoch 9/200
7/7 [==============================] - 0s 37ms/step - loss: 0.4932
Epoch 10/200
7/7 [==============================] - 0s 36ms/step - loss: 0.5037
Epoch 11/200
7/7 [==============================] - 0s 38ms/step - loss: 0.4997
Epoch 12/200
7/7 [==============================] - 0s 38ms/step - loss: 0.4969
Epoch 13/200
7/7 [==============================]

7/7 [==============================] - 0s 42ms/step - loss: 0.2143
Epoch 104/200
7/7 [==============================] - 0s 39ms/step - loss: 0.2112
Epoch 105/200
7/7 [==============================] - 0s 39ms/step - loss: 0.2008
Epoch 106/200
7/7 [==============================] - 0s 39ms/step - loss: 0.2026
Epoch 107/200
7/7 [==============================] - 0s 39ms/step - loss: 0.2141
Epoch 108/200
7/7 [==============================] - 0s 41ms/step - loss: 0.2054
Epoch 109/200
7/7 [==============================] - 0s 39ms/step - loss: 0.2098
Epoch 110/200
7/7 [==============================] - 0s 40ms/step - loss: 0.2102
Epoch 111/200
7/7 [==============================] - 0s 37ms/step - loss: 0.2072
Epoch 112/200
7/7 [==============================] - 0s 40ms/step - loss: 0.1942
Epoch 113/200
7/7 [==============================] - 0s 39ms/step - loss: 0.1898
Epoch 114/200
7/7 [==============================] - 0s 42ms/step - loss: 0.1885
Epoch 115/200
7/7 [=======================

### Rebuild model for predicting one output rather than a vector

In [11]:
#change config
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)
#load trained weights from checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate Outputs from Model

In [12]:
def Get_Prediction(model, opponent_history):
    #format history for input
    input_eval = [0 if c == '' else RPS_encode(c) for c in opponent_history]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 0.1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    #print(predicted_id)
    #print(prediction)
    return RPS_decode(predicted_id)

#print(Get_Prediction(model, ['R','R','P']))

# RPS.py
Player function

In [13]:
plays = 1000

def player(prev_play, opponent_history = [], play_history=['']):
    if len(play_history) >= plays: 
        play_history.clear()
        play_history.append('')
        print(play_history)
    guess = Get_Prediction(model, play_history)
    play_history.append(guess)
    return guess

# MAIN.PY

In [14]:
play(player, quincy, plays)
play(player, abbey, plays)
play(player, kris, plays)
play(player, mrugesh, plays)

['']
Final results: {'p1': 332, 'p2': 334, 'tie': 334}
Player 1 win rate: 49.849849849849846%
['']
Final results: {'p1': 416, 'p2': 417, 'tie': 167}
Player 1 win rate: 49.93997599039616%
['']
Final results: {'p1': 333, 'p2': 333, 'tie': 334}
Player 1 win rate: 50.0%
['']
Final results: {'p1': 250, 'p2': 498, 'tie': 252}
Player 1 win rate: 33.42245989304813%


33.42245989304813

## Saving the model

In [22]:
model.save('Models/LSTM_ImbalancedTrainingAbbey_PlayerHistoryBased_Abbey')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/LSTM_ImbalancedTrainingAbbeyKris_PlayerHistoryBased_Kris_Mrugesh\assets
